# 11/18 THIS NOTE BOOK IS OUT DATED: Preprocessing
Naive bayes will be the base model, and it will struggle with stop words and common words. For that reason, we need to preprocess the data.

preprocessing steps include
1. remove end of book
2. remove start of book
3. cleaning extra newline characters
7. tokenize words
8. Stemming
4. map authors to labels
5. tokenize to sentences
6. make (few sentence, label) pairs

X. remove accented characters to ASCII
X. Expand contractions
X. Lowercase(?) text
X. Convert numbers to words
X. Remove numbers
X. Remove stop words (not for bert)
9. Lemmatization (convert to verb root, singular nouns)(nltk package)(not for bert)
10. split data into train, validate, and test

There are a variety of python packages that will help with these steps. They are

# Table of packages here

In [1]:
# imports
import requests
import re
import pandas as pd
import os

In [2]:
os.chdir('..')
os.getcwd()


'/home/denny/Documents/mids/w266_NLP/lit-shazam'

In [3]:
# get data
from src.data.make_dataset import get_book
# from data.raw.book_urls import book_urls # not working?
book_urls = {'great_gatsby':'https://www.gutenberg.org/cache/epub/64317/pg64317.txt',
'the_sun_also_rises':'https://www.gutenberg.org/cache/epub/67138/pg67138.txt',
'a_tale_of_two_cities':'https://www.gutenberg.org/cache/epub/98/pg98.txt'}


In [4]:
data_set = {}
for title, url in book_urls.items():
    print(url)
    data_set[title] = get_book(url)

https://www.gutenberg.org/cache/epub/64317/pg64317.txt
https://www.gutenberg.org/cache/epub/67138/pg67138.txt
https://www.gutenberg.org/cache/epub/98/pg98.txt


# 1.0 Remove Bookends
Books in project gutenberg have lots of extra text at the end of the text file. Most of this is legalese and terms of use information and this is unneeded for the language modeling.

Let's look at some examples and then create a framework to remove it from the text

In [5]:
great_gatsby = data_set['great_gatsby']

In [7]:
great_gatsby[-9000:-8000]

' 1.F.3, a full refund of\r\n        any money paid for a work or a replacement copy, if a defect in the\r\n        electronic work is discovered and reported to you within 90 days of\r\n        receipt of the work.\r\n    \r\n    • You comply with all other terms of this agreement for free\r\n        distribution of Project Gutenberg™ works.\r\n    \r\n\r\n1.E.9. If you wish to charge a fee or distribute a Project\r\nGutenberg™ electronic work or group of works on different terms than\r\nare set forth in this agreement, you must obtain permission in writing\r\nfrom the Project Gutenberg Literary Archive Foundation, the manager of\r\nthe Project Gutenberg™ trademark. Contact the Foundation as set\r\nforth in Section 3 below.\r\n\r\n1.F.\r\n\r\n1.F.1. Project Gutenberg volunteers and employees expend considerable\r\neffort to identify, do copyright research on, transcribe and proofread\r\nworks not protected by U.S. copyright law in creating the Project\r\nGutenberg™ collection. Despite

In [8]:
# end pattern
print('*** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***')
end_gutenberg = '*** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***'

*** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***


In [10]:
pattern = r'\*\*\* END OF THE PROJECT GUTENBERG EBOOK [\w\d\s]+ \*\*\*'
p = re.compile(pattern)
print('the span of the found text is ', re.search(pattern, great_gatsby).span())
print('the start of the found text is ', re.search(pattern, great_gatsby).start())
print('we should remove everything after the start fo the end of book pattern')
re.search(pattern, great_gatsby).start()

the span of the found text is  (277737, 277796)
the start of the found text is  277737
we should remove everything after the start fo the end of book pattern


277737

In [11]:
# all books in project gutenberg end with 
# *** END OF THE PROJECT GUTENBERG EBOOK {Title} ***
# *** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***

import re
def remove_bookend(book:str)->str:
    """removes the extra end of the book in project gutenberg"""
    end_of_book_pattern = r'\*\*\* END OF THE PROJECT GUTENBERG EBOOK [\w\d\s]+ \*\*\*'
    match = re.search(end_of_book_pattern, book)
    if match is None:
        print('could not find project gutenberg ending')
        raise ValueError
    last_character = match.start()
    return book[:last_character]

### Unittest

In [12]:
test_book_ending ='This is the end of the book. *** END OF THE PROJECT GUTENBERG EBOOK THE the book title with number 10 ***'
test_book_no_ending = remove_bookend(test_book_ending)
assert test_book_no_ending == 'This is the end of the book. '

# 2.0 Remove book start
This is a harder problem. 
We do know that all project gutenberg books have boiler plate starting text, which we can find and remove. However, some books have table of contents, other books have introductions, preambles, etc.


In [13]:
def remove_book_start(book:str)->str:
    """removes the boiler plate beginning part of the book in project gutenberg"""
    start_of_book_pattern = r'\*\*\* START OF THE PROJECT GUTENBERG EBOOK [\w\d\s]+ \*\*\*'
    match = re.search(start_of_book_pattern, book)
    if match is None:
        print('could not find project gutenberg beginning')
        raise ValueError
    first_character = match.end()
    return book[first_character:]

## Unittest

In [14]:
test_book_start = '\ufeffThe Project Gutenberg eBook of The Great Gatsby        This ebook is for the use of anyone anywhere in the United States and  most other parts of the world at no cost and with almost no restrictions  whatsoever. You may copy it, give it away or re-use it under the terms  of the Project Gutenberg License included with this ebook or online  at www.gutenberg.org. If you are not located in the United States,  you will have to check the laws of the country where you are located  before using this eBook.    Title: The Great Gatsby      Author: F. Scott Fitzgerald    Release date: January 17, 2021 [eBook #64317]    Language: English        *** START OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***          The Great Gatsby        by      F. Scott Fitzgerald                                 Table of Contents    I  II  III  IV  V  VI  VII  VIII  IX                                    Once again                                    to                                   Zelda      Then wear the go'
test_book_no_start = remove_book_start(test_book_start)
assert test_book_no_start == '          The Great Gatsby        by      F. Scott Fitzgerald                                 Table of Contents    I  II  III  IV  V  VI  VII  VIII  IX                                    Once again                                    to                                   Zelda      Then wear the go'

In [16]:
gg = remove_book_start(great_gatsby)
gg = remove_bookend(gg)
gg[:500]

'\r\n\r\n\t\t\t   The Great Gatsby\r\n\t\t\t\t  by\r\n\t\t\t F. Scott Fitzgerald\r\n\r\n\r\n                           Table of Contents\r\n\r\nI\r\nII\r\nIII\r\nIV\r\nV\r\nVI\r\nVII\r\nVIII\r\nIX\r\n\r\n\r\n                              Once again\r\n                                  to\r\n                                 Zelda\r\n\r\n  Then wear the gold hat, if that will move her;\r\n  If you can bounce high, bounce for her too,\r\n  Till she cry “Lover, gold-hatted, high-bouncing lover,\r\n  I must have you!”\r\n\r\n  Thomas Parke d’Invilliers\r\n\r\n\r\n          '

In [17]:
for title, book in data_set.items():
    book = remove_book_start(book)
    book = remove_bookend(book)
    data_set[title] = book

# 3.0 remove unwanted characters

In [18]:
# first, remove unwanted new line and tab characters from the text
for char in ["\n", "\r", "\d", "\t", "\s\s\s"]:
    gg = gg.replace(char, " ")

In [19]:
# remove header, footer from gutenberg
gg[900:1000].split()

['and',
 'I',
 'understood',
 'that',
 'he',
 'meant',
 'a',
 'great',
 'deal',
 'more',
 'than',
 'that.',
 'In',
 'consequence,',
 'I’m',
 'inclined',
 'to',
 'reserve']

In [20]:
def remove_new_line_tabs(book):
    """remmove unwanted newlines, tabs, etc from the text"""
    for char in ["\n", "\r", "\d", "\t", "\s"]:
        book = book.replace(char, " ")
    return book


## Unittest

In [21]:
# unittest
test_book = 'hello \n I\'m trying\t to show \d the new tabs \s \r and how it gets \n broken up. \r Poetic!'
print(test_book)
test_book_ans = "hello   I'm trying  to show   the new tabs     and how it gets   broken up.   Poetic!"
assert test_book_ans == remove_new_line_tabs(test_book)
print(remove_new_line_tabs(test_book))

hello 
 and how it gets 
 Poetic!
hello   I'm trying  to show   the new tabs     and how it gets   broken up.   Poetic!


In [22]:
for title, book in data_set.items():
    print(f'processing {title}')
    data_set[title] = remove_new_line_tabs(book)

processing great_gatsby
processing the_sun_also_rises
processing a_tale_of_two_cities


# 4.0 Map authors to labels

In [23]:
id2author = {0:'Fitzgerald',1:'Hemingway',2:'Dickens'}
author2id = {value:key for (key, value) in id2author.items()}
print(author2id)

{'Fitzgerald': 0, 'Hemingway': 1, 'Dickens': 2}


# 5.0 Tokenize sentences
We'll use the sent_tokenize package from nltk. An alternative is the tf_text tokenizer.

The sent_tokenizer takes text to split into sentences and outputs the sentences as a list.



In [24]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /home/denny/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
sentences = sent_tokenize(gg)
print(f'the sentences are of {type(sentences)} and there are {len(sentences)} many sentences')

the sentences are of <class 'list'> and there are 2439 many sentences


# 8.0 Tokenize words
We use the nltk tokenizer which comes from the treebank word tokenizer.

not sure if this is actually needed, but it is required for stemming and lemmatization

In [ ]:
# 8.1 nltk tokenizer

In [ ]:
s1, s2, s3 = sentences[1], sentences[2], sentences[3]
print(s2)

In [ ]:
from nltk import tokenize
tb_tokenizer = tokenize.treebank.TreebankWordTokenizer()
token_list = tb_tokenizer.tokenize(s2)
print(token_list[0:])

In [ ]:
# 8.2 tf textvectorizer

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
tv = TextVectorization()


# 9.0 Stemming

In [ ]:
from nltk.stem import PorterStemmer
porter = PorterStemmer()
porter_tokens = [porter.stem(token) for token in token_list]
porter_tokens

# Remove punctuation?

In [ ]:
# 8.0 Lemmatization

what it does
- nouns: convert plural to singular
- verbs: convert 

why it's useful

- normalization technique for tokens
- reduces variance

# Count vectorize

# 6.0 Make (few sentences, label) pairs

Our task is `given a few sentences as an example, predict the author` and that means we need to define how many a "few" sentences is. We define this as the variable `sentences_per_example` which is an integer between 2-10. We set the default to 3. 

Then, we will take our "few sentence example" and pair that with the author in a tuple of (X, y)

In [26]:
sentences_per_example = 3
data = []
author = 'Fitzgerald'
for i in range(int(len(sentences)/sentences_per_example)):
    sentence_cluster = sentences[i*sentences_per_example:(i+1)*sentences_per_example] 
    data += [(sentence_cluster, author)]
print(i, len(data))

812 813


In [27]:
def convert_to_sentences(book, author_id, sentences_per_example=3):
    """returns a list of (sentences_per_example, author) pairs """
    sentences = sent_tokenize(book)
    total_clusters = int(len(sentences)/sentences_per_example)
    data = []
    for i in range(total_clusters):
        sentence_cluster = sentences[i*sentences_per_example:(i+1)*sentences_per_example]
        data += [(sentence_cluster, author_id)]
        
    return data
author_id = author2id['Fitzgerald']
convert_to_sentences(gg, author_id)[0]

(['          The Great Gatsby        by      F. Scott Fitzgerald                                 Table of Contents    I  II  III  IV  V  VI  VII  VIII  IX                                    Once again                                    to                                   Zelda      Then wear the gold hat, if that will move her;    If you can bounce high, bounce for her too,    Till she cry “Lover, gold-hatted, high-bouncing lover,    I must have you!”      Thomas Parke d’Invilliers                                        I    In my younger and more vulnerable years my father gave me some advice  that I’ve been turning over in my mind ever since.',
  '“Whenever you feel like criticizing anyone,” he told me, “just  remember that all the people in this world haven’t had the advantages  that you’ve had.”    He didn’t say any more, but we’ve always been unusually communicative  in a reserved way, and I understood that he meant a great deal more  than that.',
  'In consequence, I’m inclined 

In [ ]:
authors = ('Fitzgerald','Hemingway','Dickens')
for title, author in zip(data_set.keys(), authors):
    if author == 'Fitzgerald': 
        author_id = author2id[author]
        print(title, author, author_id)
        book = data_set[title]
        data = convert_to_sentences(book, author_id, sentences_per_example=3)
    if title == 'Hemingway': 
        break

In [ ]:
data[0]

In [ ]:
data[-1]

# 8.0 Tokenize words
We use the nltk tokenizer which comes from the treebank word tokenizer.

not sure if this is actually needed, but it is required for stemming and lemmatization

In [ ]:
s1, s2, s3 = data[6][0]
print(s2)

In [ ]:
from nltk import tokenize
tb_tokenizer = tokenize.treebank.TreebankWordTokenizer()
token_list = tb_tokenizer.tokenize(s2)
print(token_list[0:])

# 9.0 Stemming

In [ ]:
from nltk.stem import PorterStemmer
porter = PorterStemmer()
porter_tokens = [porter.stem(token) for token in token_list]
porter_tokens

# 8.0 Lemmatization

what it does
- nouns: convert plural to singular
- verbs: convert 

why it's useful

- normalization technique for tokens
- reduces variance

In [ ]:
# https://www.geeksforgeeks.org/python-lemmatization-approaches-with-examples/#
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
 
lemmatizer = WordNetLemmatizer()
 
# Define function to lemmatize each word with its POS tag
 
# POS_TAGGER_FUNCTION : TYPE 1
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [ ]:
sentence = 'the cat is sitting with the bats on the striped mat under many badly flying geese'
 
# tokenize the sentence and find the POS tag for each token
pos_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
 
print(pos_tagged)
#>[('the', 'DT'), ('cat', 'NN'), ('is', 'VBZ'), ('sitting', 'VBG'), ('with', 'IN'), 
# ('the', 'DT'), ('bats', 'NNS'), ('on', 'IN'), ('the', 'DT'), ('striped', 'JJ'), 
# ('mat', 'NN'), ('under', 'IN'), ('many', 'JJ'), ('flying', 'VBG'), ('geese', 'JJ')]
 
# As you may have noticed, the above pos tags are a little confusing.
 
# we use our own pos_tagger function to make things simpler to understand.
wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
print(wordnet_tagged)
#>[('the', None), ('cat', 'n'), ('is', 'v'), ('sitting', 'v'), ('with', None), 
# ('the', None), ('bats', 'n'), ('on', None), ('the', None), ('striped', 'a'), 
# ('mat', 'n'), ('under', None), ('many', 'a'), ('flying', 'v'), ('geese', 'a')]
 
lemmatized_sentence = []
for word, tag in wordnet_tagged:
    if tag is None:
        # if there is no available tag, append the token as is
        lemmatized_sentence.append(word)
    else:        
        # else use the tag to lemmatize the token
        lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
lemmatized_sentence = " ".join(lemmatized_sentence)
 
print(lemmatized_sentence)
#> the cat can be sit with the bat on the striped mat under many fly geese

In [ ]:
# # WordNet Lemmatizer
# from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
# lemmatizer = WordNetLemmatizer()
# lemma_tokens = [lemmatizer.lemmatize(token) for token in token_list]

In [ ]:
# # part of speech tagging
# pos_tag = nltk.pos_tag(token_list)
# pos_tag

In [ ]:
# # Get Lemma Tag Function
# def get_lemma_tag(pos_tag):
#     if pos_tag.startswith('J'):
#         return 'a'
#     elif pos_tag.startswith('V'):
#         return 'v'
#     elif pos_tag.startswith('N'):
#         return 'n'
#     elif pos_tag.startswith('R'):
#         return 'r'
#     elif pos_tag.startswith('PRP'):
#         return 'n'
#     elif pos_tag.startswith('DT'):
#         return 'a'
#     else:
#         return None

In [ ]:
# lemmatizer.lemmatize('programming', pos = get_lemma_tag('VBZ'))

In [ ]:
# lemmatizer.lemmatize('of', pos = get_lemma_tag('None'))

In [ ]:
# nltk.download('tagsets')
# nltk.help.upenn_tagset('IN')

In [ ]:
# ?lemmatizer.lemmatize

In [ ]:
# for token, pos in pos_tag:
#     print(token, pos)
#     print(get_lemma_tag(pos))
#     print(lemmatizer.lemmatize(token, pos=get_lemma_tag(pos)))

In [ ]:
# [lemmatizer.lemmatize(token, pos = get_lemma_tag(pos)) for (token, pos) in pos_tag]

In [ ]:
# [(token_, get_lemma_tag(pos)) for token, (token_, pos) in zip(token_list, pos_tag)]

In [ ]:
# lemmatizer = WordNetLemmatizer()
# lemma_tokens = [lemmatizer.lemmatize(token, pos = get_lemma_tag(pos_t)) for (token, (tok, pos_t)) in zip(token_list, nltk.pos_tag(token_list))]

In [ ]:
s1, s2, s3 = data[6][0]
print(s2)

In [ ]:
from nltk import tokenize
tb_tokenizer = tokenize.treebank.TreebankWordTokenizer()
token_list = tb_tokenizer.tokenize(s2)
print(token_list[0:])

In [ ]:
s2 = s2.split()
print(s2)

In [ ]:
import nltk
pos_tags = nltk.pos_tag(token_list)

# Shuffle data into train val test

In [28]:
train_frac = 0.6
val_frac = 0.1
test_frac = 0.3
assert train_frac + val_frac +test_frac == 1.0

## use numpy to shuffle data

In [29]:
import numpy as np
np.random.seed(123)
print(f'first record: {data[0]}')
np.random.shuffle(data)
print(f'\n\n\nfirst record after shuffle: {data[0]}')


first record: (['          The Great Gatsby        by      F. Scott Fitzgerald                                 Table of Contents    I  II  III  IV  V  VI  VII  VIII  IX                                    Once again                                    to                                   Zelda      Then wear the gold hat, if that will move her;    If you can bounce high, bounce for her too,    Till she cry “Lover, gold-hatted, high-bouncing lover,    I must have you!”      Thomas Parke d’Invilliers                                        I    In my younger and more vulnerable years my father gave me some advice  that I’ve been turning over in my mind ever since.', '“Whenever you feel like criticizing anyone,” he told me, “just  remember that all the people in this world haven’t had the advantages  that you’ve had.”    He didn’t say any more, but we’ve always been unusually communicative  in a reserved way, and I understood that he meant a great deal more  than that.', 'In consequence, I’m

In [30]:
total = len(data)
train = data[0:int(total*train_frac)]
val = data[int(total*train_frac):int(total*train_frac)+int(total*val_frac)]
test = data[int(total*train_frac)+int(total*val_frac):]

print(len(train), len(val), len(test), len(train)+len(val)+len(test))

487 81 245 813


In [31]:
np.random.seed(123)
def split_train_test_val(train_frac:float=0.6, val_frac:float=0.1, test_frac:float=0.3, data:list[tuple]=[])->tuple[list,list,list]:
    """takes a list of tuple examples, shuffles them, and splits them into train, val and test data sets based on fractions
    The tuples are
    ([sentences], label),
    ([sentences], label), ...
    
    Returns 3 lists, 
        train - list of tuples like ([train sentences], train label)
        val - list of tuples like ([validation sentences], validation label) 
        test - list of tuples like([test sentences], test label) 
    """
    assert train_frac + val_frac +test_frac == 1.0, 'tra, val, and test frac must sum to 1'
    
    # steps
    # 1. shuffle data
    # 2. split data into train, val and test slices
    # 3. return train, val, test
    
    total_examples = len(data)
    
    # shuffle the tuples
    np.random.shuffle(data)
    
    # slice random tuples
    train = data[0:int(total_examples*train_frac)]
    val = data[int(total_examples*train_frac):int(total_examples*train_frac)+int(total_examples*val_frac)]
    test = data[int(total_examples*train_frac)+int(total_examples*val_frac):]
    
    # prints the indecies of the train, val, and test lists. uncomment for debugging
    # print(0,int(total_examples*train_frac), int(total_examples*train_frac)+int(total_examples*val_frac), int(total_examples*train_frac)+int(total_examples*val_frac)+int(total_examples*test_frac))
    
    # print the total tuples in each set
    print(len(train), len(val), len(test), len(train)+len(val)+len(test))
    
    assert len(train)+len(val)+len(test) == total_examples, 'train, val, and test examples are wrong length'
    
    return train, val, test

train, val, test = split_train_test_val(train_frac, val_frac, test_frac, data)
train[0]

487 81 245 813


(['That was nineteen-seventeen.',
  'By the next year I had a few beaux  myself, and I began to play in tournaments, so I didn’t see Daisy very  often.',
  'She went with a slightly older crowd—when she went with anyone  at all.'],
 'Fitzgerald')

# Preprocess the data

1. for each book
2. convert text, author to (sentences, labels)
3. split to train, val, test lists

In [32]:
authors = ('Fitzgerald','Hemingway','Dickens')
train = []
val = []
test = []
for title, author in zip(data_set.keys(), authors):
    
    author_id = author2id[author]
    print(title, author, author_id)
    book = data_set[title]
    data = convert_to_sentences(book, author_id, sentences_per_example=3)
    train_, val_, test_ = split_train_test_val(train_frac, val_frac, test_frac, data)
    train += train_
    val += val_
    test += test_
    
 
 # shuffle when done
np.random.shuffle(train)
np.random.shuffle(val)
np.random.shuffle(test)

great_gatsby Fitzgerald 0
487 81 245 813
the_sun_also_rises Hemingway 1
1128 188 564 1880
a_tale_of_two_cities Dickens 2
1124 187 563 1874


In [33]:
np.random.shuffle(train)
np.random.shuffle(val)
np.random.shuffle(test)
train[0:5]

[(['“Is he all right?” Edna asked.',
   '“We’d better walk with him.”    “I’m all right,” I said.',
   '“Don’t come.'],
  1),
 (['Bill was tired after the bull-fight.',
   'So was I.',
   'We both took a bull-fight  very hard.'],
  1),
 (['Tomorrow!” Then she added  irrelevantly: “You ought to see the baby.”    “I’d like to.”    “She’s asleep.',
   'She’s three years old.',
   'Haven’t you ever seen her?”    “Never.”    “Well, you ought to see her.'],
  0),
 (['Whether or  no, the mender of roads ran, on the sultry morning, as if for his life,  down the hill, knee-high in dust, and never stopped till he got to the  fountain.',
   'All the people of the village were at the fountain, standing about  in their depressed manner, and whispering low, but showing no other  emotions than grim curiosity and surprise.',
   'The led cows, hastily brought  in and tethered to anything that would hold them, were looking stupidly  on, or lying down chewing the cud of nothing particularly repaying thei

In [34]:
print(f'training examples: {len(train)}\nvalidation examples: {len(val)}\ntest examples: {len(test)}')

training examples: 2739
validation examples: 456
test examples: 1372


In [35]:
train[0]

(['“Is he all right?” Edna asked.',
  '“We’d better walk with him.”    “I’m all right,” I said.',
  '“Don’t come.'],
 1)

# Count Vectorize
In this section, the words for each example will be converted to a numeric value and stored as a sparse matrix in CSR format (see scipy.sparse.csr_matrix for more info)

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

In [41]:
s = 'hi my name is Paul'
t = 'hi Paul, my name is Denny'
cv = CountVectorizer()

X = cv.fit_transform([s,t])
print(cv.get_feature_names_out())
print(X.toarray())


['denny' 'hi' 'is' 'my' 'name' 'paul']
[[0 1 1 1 1 1]
 [1 1 1 1 1 1]]


In [37]:
# get vocabulary
# vocabulary = 
v = set()
for sent in train:
    v.update(set(sent))
len(v)

TypeError: unhashable type: 'list'

In [53]:
train_examples = [pair[0] for pair in train]
train_labels = [pair[1] for pair in train]

In [58]:
''.join(train_examples[0])

'“Is he all right?” Edna asked.“We’d better walk with him.”    “I’m all right,” I said.“Don’t come.'

In [60]:
cv = CountVectorizer()
cv.fit_transform([''.join(sentences) for sentences in train_examples])

<2739x10790 sparse matrix of type '<class 'numpy.int64'>'
	with 110578 stored elements in Compressed Sparse Row format>

In [46]:
t = s.split()
set(t)
train[0][0]

['“Is he all right?” Edna asked.',
 '“We’d better walk with him.”    “I’m all right,” I said.',
 '“Don’t come.']

# save the data

In [ ]:
import pickle
with open('data.pkl', 'wb') as f:
    pickle.dump([train, val, test], f)